In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

import parallelism
parallelism.set_max_num_tbb_threads(24)
parallelism.set_hessian_assembly_num_threads(8)
parallelism.set_gradient_assembly_num_threads(8)

### Initialization

In [ ]:
handleBoundary = False

In [ ]:
name = 'hemisphere_5t'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=handleBoundary, handlePivots = True)

In [ ]:
use_pin = True

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
um_1 = pickle.load(gzip.open('2022_01_25_16_43_saddle_5t/saddle_5t_input_equilibrium_2022_01_25_16_43_target_height_factor_5.0.pkl.gz', 'r'))

In [ ]:
um_2 = pickle.load(gzip.open('2022_01_25_14_23_hemisphere_5t/hemisphere_5t_tsf_equilibrium_2022_01_25_14_23_target_height_factor_5.0.pkl.gz', 'r'))

In [ ]:
um_3 = pickle.load(gzip.open('2022_01_25_16_43_saddle_5t/saddle_5t_force_equilibrium_2022_01_25_16_43_target_height_factor_5.0.pkl.gz', 'r'))

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

rod_colors = get_color_field(curr_um, input_data)

import mesh
mv1 = linkage_vis.LinkageViewerWithSurface(um_1, target_mesh, width=1024, height=600)
set_surface_view_options(mv1, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
mv1.averagedMaterialFrames = True
mv1.showScalarField(rod_colors)
mv1.show()

In [ ]:
configure_umbrella_true_equlibrium(um_1, thickness, target_height_multiplier)
def eqm_callback(prob, i):
    if (i % 2 == 0):
        mv1.showScalarField(rod_colors)
with so(): results = umbrella_mesh.compute_equilibrium(um_1, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

In [ ]:
import force_analysis
with so(): importlib.reload(force_analysis)

In [ ]:
force_analysis.UmbrellaForceAnalysis(um_1)
v1 = force_analysis.UmbrellaForceFieldVisualization(um_1)
v1.show()

In [ ]:
import mesh
mv2 = linkage_vis.LinkageViewerWithSurface(um_2, target_mesh, width=1024, height=600)
set_surface_view_options(mv2, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
mv2.averagedMaterialFrames = True
mv2.showScalarField(rod_colors)
mv2.show()

In [ ]:
configure_umbrella_true_equlibrium(um_2, thickness, target_height_multiplier)
def eqm_callback(prob, i):
    if (i % 2 == 0):
        mv2.showScalarField(rod_colors)
with so(): results = umbrella_mesh.compute_equilibrium(um_2, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

In [ ]:
import force_analysis
with so(): importlib.reload(force_analysis)

In [ ]:
force_analysis.UmbrellaForceAnalysis(um_2)
v2 = force_analysis.UmbrellaForceFieldVisualization(um_2)
v2.show()

In [ ]:
import mesh
mv3 = linkage_vis.LinkageViewerWithSurface(um_3, target_mesh, width=1024, height=600)
set_surface_view_options(mv3, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
mv3.averagedMaterialFrames = True
mv3.showScalarField(rod_colors)
mv3.show()

In [ ]:
configure_umbrella_true_equlibrium(um_2, thickness, target_height_multiplier)

def eqm_callback(prob, i):
    if (i % 2 == 0):
        mv3.showScalarField(rod_colors)
with so(): results = umbrella_mesh.compute_equilibrium(um_3, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

In [ ]:
import force_analysis
with so(): importlib.reload(force_analysis)

In [ ]:
force_analysis.UmbrellaForceAnalysis(um_3)
v3 = force_analysis.UmbrellaForceFieldVisualization(um_3)
v3.show()

In [ ]:
import time
time_stamp = time.strftime("%Y_%m_%d_%H_%M")
import os
output_folder = '{}_{}'.format(time_stamp, name)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  


In [ ]:
importlib.reload(pipeline_helper)

In [ ]:

input_pickle_path = '{}/{}_input_equilibrium_{}_target_height_factor_{}.pkl.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

input_rendering_path = '{}/{}_input_equilibrium_{}_rendering_output_{}.json.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

pipeline_helper.save_data(um_1, input_pickle_path, input_rendering_path, input_path, False, handleBoundary)


In [ ]:

tsf_pickle_path = '{}/{}_tsf_equilibrium_{}_target_height_factor_{}.pkl.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

tsf_rendering_path = '{}/{}_tsf_equilibrium_{}_rendering_output_{}.json.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

pipeline_helper.save_data(um_2, tsf_pickle_path, tsf_rendering_path, input_path, False, handleBoundary)


In [ ]:

force_pickle_path = '{}/{}_force_equilibrium_{}_target_height_factor_{}.pkl.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

force_rendering_path = '{}/{}_force_equilibrium_{}_rendering_output_{}.json.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

pipeline_helper.save_data(um_3, force_pickle_path, force_rendering_path, input_path, False, handleBoundary)


In [ ]:
import force_vector_visualization_helper
importlib.reload(force_vector_visualization_helper)

force_vector_visualization_helper.write_force_vector_visualization_file([tsf_pickle_path, force_pickle_path], ['{}/{}_tsf'.format(output_folder, name), '{}/{}_force'.format(output_folder, name)])

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False
        
optimizer = umbrella_optimization.UmbrellaOptimization(um_3, opt_opts, 2.5, -1, False, fixedVars)

optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1
optimizer.iota = 0

rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
fabrication_path = '{}/{}_force_equilibrium_{}_target_height_factor_{}.json.gz'.format(output_folder, name, time_stamp, target_height_multiplier)

In [ ]:
import importlib, load_jsondata
importlib.reload(load_jsondata)
load_jsondata.update_optimized_json(input_path, rest_height_optimizer.params(), output_json_path = fabrication_path, optim_spacing_factor = target_height_multiplier, handleBoundary = False)
